This notebook includes (will include) basic first visualisation of the cycling data.

Data parameters:
- Time
- Cell potential (V)
- Current (mA)
- Temperature
- Cycle #

In [33]:
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go

In [34]:
#Get data into data frames

file_paths = [
    ('Data/NX001/A3_NX001_2108_CA4.txt'),
    ('Data/NX001/A3_NX001_3107_CA4.txt'),
    ('Data/NX002/A5_NX002_2108_CA5.txt'),
    ('Data/NX002/A5_NX002_3107_CA5.txt'),
    ('Data/NX006/A3_NX006_2108_CA3.txt'),
    ('Data/NX006/A3_NX006_3107_CA3.txt'),
    ('Data/RS001/A1_RS001_2108_CA1.txt'),
    ('Data/RS001/A1_RS001_3107_CA1.txt'),
    ('Data/RS006/A1_RS006_2108_CA2.txt'),
    ('Data/RS006/A1_RS006_3107_CA2.txt')
]

all_data = []

for file_path in file_paths:
    columns = ['time/s', 'Ecell/V', 'I/mA', 'Temperature/°C', 'cycle number']

    df = pd.read_csv(file_path, sep='\t', usecols=columns, encoding='ISO-8859-1')
    battery_name = '_'.join(os.path.basename(file_path).split('_')[-3:-1])
    all_data.append((df,battery_name))

df_NX001_2108, battery_name_NX001_2108 = all_data[0]
df_NX001_3107, battery_name_NX001_2401 = all_data[1]
df_NX002_2108, battery_name_NX002_2108 = all_data[2]
df_NX002_3107, battery_name_NX002_2108 = all_data[3]
df_NX006_2108, battery_name_NX006_2108 = all_data[4]
df_NX006_3107, battery_name_NX006_2401 = all_data[5]
df_RS001_2108, battery_name_RS001_2108 = all_data[6]
df_RS001_3107, battery_name_RS001_2108 = all_data[7]
df_RS006_2108, battery_name_RS006_2108 = all_data[8]
df_RS006_3107, battery_name_RS006_2401 = all_data[9]

In [35]:
def plot_cycle(df, battery_name, cycle_number):
    
    cycle_data = df[df['cycle number'] == cycle_number]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=cycle_data['time/s'], y=cycle_data['Ecell/V'], mode='lines', name='Ecell/V'))
    fig.add_trace(go.Scatter(x=cycle_data['time/s'], y=cycle_data['I/mA'], mode='lines', name='I/mA'))
    fig.add_trace(go.Scatter(x=cycle_data['time/s'], y=cycle_data['Temperature/°C'], mode='lines', name='Temperature/°C'))

    fig.update_layout(title=f'Cycle {cycle_number} for Battery {battery_name}', xaxis_title='Time (s)', yaxis_title='Value')

    fig.show()

    fig.update_layout(
        title=f'Cycle {cycle_number} for Battery {battery_name}',
        xaxis_title='Time (s)',
        yaxis_title='Value',
        template='plotly_white'
    )

    fig.show()


In [36]:
for df, battery_name in all_data:
    plot_cycle(df, battery_name, 1)

### Capacity
Calculate the capacity by ompute the time interval between successive measurements, and then integrate the current wrt to time. I've used `numpy.trapz` trapezoidal rule mathematical function.

In [37]:
#Calculate capacity and add a column to the original df

def add_capacity(df):
    grouped = df.groupby('cycle number')
    capacities = []

    for cycle, group in grouped:
        times = group['time/s'].values
        currents = group['I/mA'].values
        
        charge = np.trapz(currents, times)
        capacity_mAh = charge / 3600.0
        capacities.append((cycle, capacity_mAh))
    
    capacity_df = pd.DataFrame(capacities, columns=['cycle number', 'capacity/mAh'])
    df = df.merge(capacity_df, on='cycle number', how='left')
    
    return df


In [38]:
all_data_c = []

for df, battery_name in all_data:
    df = add_capacity(df)
    all_data_c.append((df,battery_name))


## Plotting capacity against cycles

In [ ]:
for df, battery_name in all_data_c:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['cycle number'], y=df['capacity/mAh'], mode='lines+markers', name=battery_name))
    fig.update_layout(title=f'Battery Capacity Against Number of Cycles for Battery {battery_name}',
                      xaxis_title='Cycle Number',
                      yaxis_title='Capacity (mAh)')
    fig.show()

In [39]:
#Don't use these fns... too slow

def resample(df, time_column, rule):
    df[time_column] = pd.to_datetime(df[time_column], unit='s')
    return df.resample(rule, on=time_column).mean().reset_index()

def plot_all_cycles(df, battery_name):
    #sample the data as otherwise it takes forever to plot

    df_sampled = resample(df, 'time/s', '30s')
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df['time/s'], y=df_sampled['Ecell/V'], mode='lines', name='Ecell/V'))
    fig.add_trace(go.Scatter(x=df['time/s'], y=df_sampled['I/mA'], mode='lines', name='I/mA'))
    fig.add_trace(go.Scatter(x=df['time/s'], y=df_sampled['Temperature/°C'], mode='lines', name='Temperature/°C'))

    cycles = df['cycle number'].unique()

    # Add shaded areas for each cycle
    for cycle in cycles:
        cycle_data = df[df['cycle number'] == cycle]
        cycle_start = cycle_data['time/s'].min()
        cycle_end = cycle_data['time/s'].max()

        fig.add_shape(
            type='rect',
            x0=cycle_start,
            x1=cycle_end,
            y0=0,
            y1=1, 
            yref='paper',
            fillcolor='LightSlateGrey' if cycle % 2 == 0 else 'LightSkyBlue',
            opacity=0.3,
            line_width=0,
        )

    # Customize layout
    fig.update_layout(
        title=f'Battery {battery_name}',
        xaxis_title='Time (s)',
        yaxis_title='Value',
        template='plotly_white',
        shapes=[],
    )

    # Show the plot
    fig.show()